In [1]:
# Required modules

import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

2024-04-12 14:30:13.206208: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-12 14:30:13.296304: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/openmpi-4.1.2-4a/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/python-3.9.9-jh/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/tcl-8.6.11-d4/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/sqlite-3.37.1-6s/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/zlib-1.2.11-2y/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_

In [2]:
# Create an instance of ImageDataGenerator for data augmentation (optional)

train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize the images
#     rotation_range=20,  # Data augmentation: Random rotations
#     width_shift_range=0.2,  # Data augmentation: Random horizontal shifts
#     height_shift_range=0.2,  # Data augmentation: Random vertical shifts
#     shear_range=0.2,  # Data augmentation: Random shearing
#     zoom_range=0.2,  # Data augmentation: Random zooming
    horizontal_flip=True,  # Data augmentation: Random horizontal flips
    fill_mode='nearest',  # Strategy for filling in newly created pixels
    validation_split=0.2  # Splitting the data: 80% for training, 20% for validation
)

# Assuming your dataset directory is "/path/to/dataset"
train_generator = train_datagen.flow_from_directory(
    './Data/images_original/',
    target_size=(432, 288),  # Resize images to 150x150
    batch_size=32,
    class_mode='categorical',  # For multi-class classification
    subset='training'  # Specify this is training data
)

validation_generator = train_datagen.flow_from_directory(
    './Data/images_original/',
    target_size=(432, 288),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Specify this is validation data
)

Found 800 images belonging to 10 classes.
Found 199 images belonging to 10 classes.


In [3]:
# A 3 layer Conv model

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(432, 288, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')  # 10 classes
])

2024-04-12 14:30:16.928441: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/openmpi-4.1.2-4a/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/python-3.9.9-jh/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/tcl-8.6.11-d4/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/sqlite-3.37.1-6s/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/zlib-1.2.11-2y/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-9.3.0/gcc-10.3.0-ya/lib64:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-9.3.0/gcc-10.3.0-ya/lib
2024-04-12 14:30:16.928481: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-04-12 14:30:16.928507: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel drive

In [4]:
# Compile the model

optims = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Model Training

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // validation_generator.batch_size
)

Epoch 1/50
25/25 [==============================] - 47s 2s/step - loss: 3.9647 - accuracy: 0.1175 - val_loss: 2.2226 - val_accuracy: 0.2500
Epoch 2/50
25/25 [==============================] - 47s 2s/step - loss: 2.1474 - accuracy: 0.2163 - val_loss: 2.0106 - val_accuracy: 0.2292
Epoch 3/50
25/25 [==============================] - 46s 2s/step - loss: 1.8319 - accuracy: 0.3688 - val_loss: 1.7282 - val_accuracy: 0.3854
Epoch 4/50
25/25 [==============================] - 47s 2s/step - loss: 1.5477 - accuracy: 0.4313 - val_loss: 1.5682 - val_accuracy: 0.4375
Epoch 5/50
25/25 [==============================] - 47s 2s/step - loss: 1.3083 - accuracy: 0.5263 - val_loss: 1.5511 - val_accuracy: 0.4844
Epoch 6/50
25/25 [==============================] - 47s 2s/step - loss: 1.1356 - accuracy: 0.6162 - val_loss: 1.4669 - val_accuracy: 0.5052
Epoch 7/50
25/25 [==============================] - 47s 2s/step - loss: 0.9507 - accuracy: 0.6625 - val_loss: 1.5841 - val_accuracy: 0.5000
Epoch 8/50
25/25 [==

In [ ]:
# Evaluate the model

val_loss, val_acc = model.evaluate(validation_generator)
print(f'Validation loss: {val_loss}, Validation accuracy: {val_acc}')